In [ ]:
#in the name of GOD#

In [3]:
import pandas as pd
import torch
from sklearn.preprocessing import MinMaxScaler

In [2]:
!git clone https://github.com/cloner174/Informer2020.git

# DO NOT REMOVE THESE #
train_size = None# DO NOT CHANGE THESE #
test_size = None# DO NOT CHANGE THESE #
seq_len = None# DO NOT CHANGE THESE #
# DO NOT CHANGE THE VALUES ABOVE #

Cloning into 'Informer2020'...
remote: Enumerating objects: 637, done.
remote: Counting objects: 100% (283/283), done.
remote: Compressing objects: 100% (95/95), done.
remote: Total 637 (delta 239), reused 188 (delta 188), pack-reused 354
Receiving objects: 100% (637/637), 6.50 MiB | 6.07 MiB/s, done.
Resolving deltas: 100% (376/376), done.


In [4]:
# Run THis for Scaling OutSide the Model #

df = pd.read_csv('/content/Informer2020/input/data.csv')

date_col_name = 'date'
target_col_name = 'Close'
features = 'MS'

df_columns = list(df.columns)

df_columns.remove(date_col_name)
df_columns.remove(target_col_name)

df_raw = df[[date_col_name]+df_columns+[target_col_name]]
df_columns.insert(0, 'date')
df_columns.append(target_col_name)

df_raw = df_raw.set_axis(df_columns, axis=1)


In [5]:
test_size = 0.2
train_size = 0.90-test_size
seq_len = 30

num_train = int(len(df_raw)*train_size)
num_test = int(len(df_raw)*test_size)
num_vali = len(df_raw) - num_train - num_test
border1s = [0, num_train-seq_len, len(df_raw)-num_test-seq_len]
border2s = [num_train, num_train+num_vali, len(df_raw)]

print("Start and End Index of Train data: ", border1s[0], '->', border1s[1])
print("\n")
print("Start and End Index of Test data: ", border2s[0], '->', border2s[1])


Start and End Index of Train data:  0 -> 2553


Start and End Index of Test data:  2583 -> 2953


In [6]:
if features=='M' or features=='MS':
    cols_data = df_raw.columns[1:]
    df_data = df_raw[cols_data]
elif features=='S':
    df_data = df_raw[[target_col_name]]

In [7]:
#range of fit and transform when you do not whant to change the target :
#     df_columns.index('date') + 1 ,  df_columns.index('Close')


scalers_values_archive = {}
scalers_archive = []
data_copy = df_data.copy()
print("Before Scale : \n", data_copy)

for any_col in df_data.columns :

    scaler = MinMaxScaler()
    train_data = df_data[[any_col]][border1s[0]:border2s[0]]
    col_data = df_data[[any_col]]
    scaler.fit(train_data.values)
    scalers_values_archive[any_col] = {
        'data_min_': torch.tensor(scaler.data_min_, dtype=torch.float32),
        'data_max_': torch.tensor(scaler.data_max_, dtype=torch.float32),
        'scale_': torch.tensor(scaler.scale_, dtype=torch.float32),
        'min_': torch.tensor(scaler.min_, dtype=torch.float32)
    }
    scalers_archive.append([any_col, scaler])
    data_copy[any_col] = scaler.transform(col_data.values)


print("After Scale : \n", data_copy)


Before Scale : 
        Open   High    Low  Final   Volume  Close
0     26000  26000  24885  24885     4520  24885
1     24885  24885  24885  24885       45  24885
2     25000  25000  25000  25000     2000  25000
3     25003  25003  25003  25003       70  25003
4     25010  25010  25010  25010      120  25010
...     ...    ...    ...    ...      ...    ...
3686  14500  15000  14320  14690  6333267  14640
3687  14890  15000  14350  14680  2392003  14550
3688  14220  14650  14140  14600  1470757  14210
3689  14530  14800  14200  14580  1910602  14490
3690  14600  14600  14230  14550   748934  14300

[3691 rows x 6 columns]
After Scale : 
           Open      High       Low     Final    Volume     Close
0     0.541841  0.514722  0.516310  0.491133  0.000349  0.490458
1     0.516299  0.490458  0.516310  0.491133  0.000003  0.490458
2     0.518933  0.492960  0.518944  0.493632  0.000154  0.492960
3     0.519002  0.493025  0.519013  0.493698  0.000005  0.493025
4     0.519162  0.493178  0.5

In [8]:
scalers_values_archive['Close'], "\n",scalers_archive[-1]

({'data_min_': tensor([2347.]),
  'data_max_': tensor([48300.]),
  'scale_': tensor([2.1761e-05]),
  'min_': tensor([-0.0511])},
 '\n',
 ['Close', MinMaxScaler()])

In [9]:
data_copy.insert(0, 'date', df_raw[['date']])
data_copy
#data_copy.to_csv('/content/Informer2020/input/data.csv', index=False)

,date,Open,High,Low,Final,Volume,Close
0,2001-03-25,0.541841,0.514722,0.516310,0.491133,0.000349,0.490458
1,2001-03-26,0.516299,0.490458,0.516310,0.491133,0.000003,0.490458
2,2001-04-08,0.518933,0.492960,0.518944,0.493632,0.000154,0.492960
3,2001-04-10,0.519002,0.493025,0.519013,0.493698,0.000005,0.493025
4,2001-04-15,0.519162,0.493178,0.519174,0.493850,0.000009,0.493178
...,...,...,...,...,...,...,...
3686,2023-11-04,0.278400,0.275347,0.274293,0.269570,0.488644,0.267512
3687,2023-11-05,0.287334,0.275347,0.274981,0.269353,0.184555,0.265554
3688,2023-11-06,0.271986,0.267730,0.270170,0.267614,0.113476,0.258155
3689,2023-11-07,0.279087,0.270994,0.271544,0.267180,0.147413,0.264248


In [ ]:
# End Of scaling Process #

In [10]:
%cd 'Informer2020'

/content/Informer2020


In [11]:
import os
import sys
import torch
import numpy as np
from utils.tools import dotdict

from exp.exp_informer import Exp_Informer

args = dotdict()



args.root_path = '/content/Informer2020/input/'

args.data_path = 'data.csv'

args.model = 'informer'                      #  options: [ informer, informerstack-NOTSUREYET!-, informerlight(TBD)-NOTSUREYET!-]

args.data = 'custom'

args.features = 'MS'

args.target = 'Close'

args.freq = 'b'

args.checkpoints = './informer_checkpint' # location of model checkpoints

args.seq_len = seq_len if seq_len is not None else 1*5*6

args.label_len = 1*1
args.pred_len = 1*1

args.enc_in  =  6         		   # Encoder input size
args.dec_in  =  6          		  # Decoder input size
args.c_out   =  1            		 # Output size

args.factor  =  1            		  # Probsparse attn factor

args.d_model = 512            		 # Dimension of model
args.n_heads = 16            		   # 16خوب
args.e_layers = 8          		 # num of encoder layers
args.d_layers = 8           		# num of decoder layers
args.d_ff    =  2048         		# dimension of fcn in model
args.s_layers = '3,2,1'			#help='num of stack encoder layers'

args.dropout  =  0.01         # dropout

args.attn = 'full'                  # attention used in encoder, options:[prob, full]
args.embed = 'learned'              # time features encoding, options:[timeF, fixed, learned]
args.activation = 'ReLU6'            # activation

args.distil = True                      # whether to use distilling in encoder
args.output_attention = False           # whether to output attention in ecoder
args.mix = True

args.padding = 0

args.batch_size = 16
args.learning_rate = 0.00005

args.loss = 'mse'
args.lradj = 'type1'

args.use_amp = False                    # whether to use automatic mixed precision training

args.num_workers = 2

args.itr = 1

args.train_epochs = 10

args.patience = 5

args.des = 'exp'			#default='test',help='exp description'

args.use_gpu = True if torch.cuda.is_available() else False

args.gpu = 0
args.use_multi_gpu = False
args.devices = '0,1,2,3'

args.do_predict = None			# help='whether to predict unseen future dat



#    #    #    new#    #

args.test_size = test_size if test_size is not None else 0.2

args.kind_of_optim = 'Adam'               # optimizer to use,options:AdamW|SparseAdam|RMSprop|AdagradRAdam|NAdam|LBFGS|AdamaxAdadelta|Adam |SGD | ASGD

args.inverse=False

args.scale=False

args.kind_of_scaler = 'MinMax'                       #   Standard   or   MinMax

args.take_data_instead_of_reading = True            # Defualt to False.. if it is True, you should provide direct_data!

args.shuffle_for_test  =  False
args.shuffle_for_pred  =  False
args.shuffle_for_train  =  True

args.criter =  'MAE'       # lose function options: WMAPE|SMAPE|MAE|RMSE|QuantileLoss|HuberLoss|PinballLoss

#    #    #    new#    #



args.use_gpu = True if torch.cuda.is_available() and args.use_gpu else False

args.s_layers = [int(s_l) for s_l in args.s_layers.replace(' ','').split(',')]
args.detail_freq = args.freq
args.freq = args.freq[-1:]

#if args.use_gpu and args.use_multi_gpu:
#    args.devices = args.devices.replace(' ','')
#    device_ids = args.devices.split(',')
#    args.device_ids = [int(id_) for id_ in device_ids]
#    args.gpu = args.device_ids[0]


print('Args in experiment:')
print(args)

Exp = Exp_Informer


This is The Enhanced Version of Orginal code , Written in 2024 
Args in experiment:
{'root_path': '/content/Informer2020/input/', 'data_path': 'data.csv', 'model': 'informer', 'data': 'custom', 'features': 'MS', 'target': 'Close', 'freq': 'b', 'checkpoints': './informer_checkpint', 'seq_len': 30, 'label_len': 1, 'pred_len': 1, 'enc_in': 6, 'dec_in': 6, 'c_out': 1, 'factor': 1, 'd_model': 512, 'n_heads': 16, 'e_layers': 8, 'd_layers': 8, 'd_ff': 2048, 's_layers': [3, 2, 1], 'dropout': 0.01, 'attn': 'full', 'embed': 'learned', 'activation': 'ReLU6', 'distil': True, 'output_attention': False, 'mix': True, 'padding': 0, 'batch_size': 16, 'learning_rate': 5e-05, 'loss': 'mse', 'lradj': 'type1', 'use_amp': False, 'num_workers': 2, 'itr': 1, 'train_epochs': 10, 'patience': 5, 'des': 'exp', 'use_gpu': False, 'gpu': 0, 'use_multi_gpu': False, 'devices': '0,1,2,3', 'do_predict': None, 'test_size': 0.2, 'kind_of_optim': 'Adam', 'inverse': False, 'scale': False, 'kind_of_scaler': 'MinMax', 'take_d

In [ ]:

for ii in range(args.itr):
    # setting record of experiments
    setting = '{}_{}_ft{}_sl{}_ll{}_pl{}_dm{}_nh{}_el{}_dl{}_df{}_at{}_fc{}_eb{}_dt{}_mx{}_{}_{}'.format(args.model, args.data, args.features,
                args.seq_len, args.label_len, args.pred_len,
                args.d_model, args.n_heads, args.e_layers, args.d_layers, args.d_ff, args.attn, args.factor,
                args.embed, args.distil, args.mix, args.des, ii)

    exp = Exp(args, data_copy) # set experiments
    print('>>>>>>>start training : {}>>>>>>>>>>>>>>>>>>>>>>>>>>'.format(setting))
    exp.train(setting)

    print('>>>>>>>testing : {}<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<'.format(setting))
    exp.test(setting)

    if args.do_predict:
        print('>>>>>>>predicting : {}<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<'.format(setting))
        exp.predict(setting, True)

    torch.cuda.empty_cache()

Use CPU
>>>>>>>start training : informer_custom_ftMS_sl30_ll1_pl1_dm512_nh16_el8_dl8_df2048_atfull_fc1_eblearned_dtTrue_mxTrue_exp_0>>>>>>>>>>>>>>>>>>>>>>>>>>
train 2553
val 370
test 738


In [ ]:
#exp.train_losses
#exp.test_losses

In [159]:
trues = np.load('/content/Informer2020/results/informer_custom_ftMS_sl10_ll1_pl1_dm512_nh16_el8_dl8_df2048_atfull_fc1_eblearned_dtTrue_mxTrue_exp_0/true.npy')
preds = np.load('/content/Informer2020/results/informer_custom_ftMS_sl10_ll1_pl1_dm512_nh16_el8_dl8_df2048_atfull_fc1_eblearned_dtTrue_mxTrue_exp_0/pred.npy')

In [160]:
print(scalers_archive[-1])
scaler = scalers_archive[-1][1]

['Close', MinMaxScaler()]


In [161]:
tt = []
pp = []

for any_ in scaler.inverse_transform(trues.reshape(-1,1)).reshape(-1):
  tt.append(round(any_))

for any_ in scaler.inverse_transform(preds.reshape(-1,1)).reshape(-1):
  pp.append(round(any_))

t = pd.Series(tt)
p = pd.Series(pp)

result = pd.concat([t,p], axis=1)
result = result.set_axis(['True', 'Predict'], axis=1)
print(result)

      True  Predict
0    65510    40612
1    68001    39504
2    72510    40443
3    33620    36996
4    35300    37003
..     ...      ...
731  13710    22766
732  14300    22139
733  14640    20051
734  14550    19842
735  14210    21293

[736 rows x 2 columns]


In [ ]:
#done

In [108]:
preds.reshape(-1)[:50]

array([0.98960656, 0.83518106, 0.9768787 , 0.87988937, 0.83912545,
       0.76452976, 0.86802346, 0.7796083 , 0.7027675 , 0.85203534,
       0.77093005, 0.8145493 , 0.8430747 , 0.7506754 , 0.8032037 ,
       0.66221505, 0.87651265, 0.632769  , 0.72353023, 0.6644593 ,
       0.7103992 , 0.8070298 , 0.7336649 , 0.65453744, 0.8173029 ,
       0.6567246 , 0.7111692 , 0.70654744, 0.7357852 , 0.5764162 ,
       0.6217591 , 0.64224666, 0.64289564, 0.57475656, 0.66826415,
       0.7096961 , 0.6165802 , 0.5982239 , 0.6449774 , 0.6558666 ,
       0.6689005 , 0.6850541 , 0.7874504 , 0.6216084 , 0.6962758 ,
       0.7006822 , 0.75089395, 0.78326863, 0.7268329 , 0.76165956],
      dtype=float32)

In [109]:
trues.reshape(-1)[:50]

array([1.374513  , 1.4287206 , 1.5268426 , 0.6805432 , 0.7171022 ,
       0.75540227, 0.7956608 , 0.8378778 , 0.882271  , 0.92884034,
       0.9762801 , 1.0276369 , 1.0800818 , 1.1066307 , 1.162775  ,
       1.1022785 , 1.0485278 , 1.0300307 , 1.0169739 , 1.0987966 ,
       1.0413465 , 1.0324244 , 1.0047874 , 0.9882489 , 0.95343065,
       0.94167954, 0.90990794, 0.87813634, 0.92775226, 0.9717102 ,
       0.99347156, 0.93036366, 0.8998977 , 0.8594216 , 0.8215568 ,
       0.7863034 , 0.793267  , 0.75627273, 0.734729  , 0.72232497,
       0.7203665 , 0.7062216 , 0.690771  , 0.6831545 , 0.67249143,
       0.73603463, 0.75279087, 0.71514374, 0.69882274, 0.6936    ],
      dtype=float32)

In [10]:
def inverser_from_numpy(data, scaler):
  return scaler.inverse_transform(data)

In [ ]:
def inverser_from_torch(data,min_,scale_, data_min_,data_max_):
  min_ = min_.to(data.device)
  scale_ = scale_.to(data.device)
  data_min_ = data_min_.to(data.device)
  data_max_ = data_max_.to(data.device)
  inversed_data = (data - min_) / scale_ * (data_max_ - data_min_) + data_min_